In [99]:
import json 
import pandas as pd
import numpy as np 
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,15)

### Load the data from JSON and put it into a dataframe

In [193]:
with open('permissions.json','r') as f:
     apps = json.loads(f.read())

In [194]:
flat_app = {}
for app in apps['results']:
    flat_app[app['appId']] = {a:app[a] for a in ['title','playstoreUrl','permissions']}

In [195]:
df = pd.DataFrame(data = list(flat_app.values()), index=list(flat_app.keys()))
df['permissions'] = [a['results'] for a in df['permissions']]

In [197]:
for index, permissions in zip(df['permissions'].index, df['permissions'].values):
    for permission in permissions:
       df.loc[index, permission['permission']] = True
df.fillna(False, inplace=True)
df['cnt'] = np.sum(df.drop(['permissions', 'playstoreUrl','title'], axis=1), axis=1)
df = df.sort_values('cnt', ascending=False)

In [179]:
df.head(2)

,permissions,playstoreUrl,title,approximate location (network-based),precise location (GPS and network-based),read the contents of your USB storage,modify or delete the contents of your USB storage,view Wi-Fi connections,read phone status and identity,view network connections,...,bind to an accessibility service,delete other apps' data,read historical network usage,control media playback and metadata access,send package removed broadcast,manage document storage,read terms you added to the dictionary,add words to user-defined dictionary,cnt,privacy_url
com.oneapp.max.cleaner.booster,"[{'permission': 'retrieve running apps', 'desc...",https://play.google.com/store/apps/details?id=...,MAX Cleaner - Phone Cleaner & Antivirus,True,True,True,True,True,True,True,...,True,True,True,False,False,False,False,False,124.0,http://oneappessentials.com/privacypolicy.html
com.psafe.msuite,"[{'permission': 'retrieve running apps', 'desc...",https://play.google.com/store/apps/details?id=...,"dfndr security: antivirus, anti-hacking & cleaner",True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,98.0,http://www.psafe.com/privacy-policy/


In [171]:
from bs4 import BeautifulSoup
import requests 

def get_privacy_url(playstoreUrl):
    html = requests.get(playstoreUrl).text
    soup = BeautifulSoup(html, 'html.parser')
    link =  [t.attrs['href'] for t in soup.find_all('a') 
                      if ('href' in t.attrs.keys()) & (t.text == 'Privacy Policy')]
    if len(link):
        return link[0]
    else:
        return ''
df['privacy_url'] = [get_privacy_url(row.playstoreUrl) for index, row in df.iterrows()]

### Apps that want to know your precise location

In [117]:
df[df['precise location (GPS and network-based)']].title

com.oneapp.max.cleaner.booster                       MAX Cleaner - Phone Cleaner & Antivirus
com.psafe.msuite                           dfndr security: antivirus, anti-hacking & cleaner
com.enflick.android.TextNow                                      TextNow - free text + calls
com.facebook.katana                                                                 Facebook
com.ehawk.antivirus.applock.wifi           Virus Cleaner ( Hi Security ) - Antivirus, Boo...
com.facebook.orca                                   Messenger – Text and Video Chat for Free
com.whatsapp                                                              WhatsApp Messenger
com.apusapps.browser                         APUS Browser - Fast Download & Private & Secure
com.ubercab                                                                             Uber
com.amazon.mShop.android.shopping                                            Amazon Shopping
com.instagram.android                                                 

So.... 'Snake VS Block' wants to know where you are... Can we find their privacy policy to see what they might want that for?

In [172]:
df.loc['com.bentostudio.ballsvsblocks'].privacy_url

'http://www.voodoo.io'

Hmm, that's kind of useless for automatically parsing - the URL doesn't link to the privacy policy itself but to the website of the developers. For completeness, below is how they're going to use the data from http://www.voodoo.io/

4 How do we use the information that we have about you?

We, our suppliers and group companies may use the information we collect about you for a number of purposes. These include providing our games and other services to you, optimising the games and services we provide, enabling in-app purchases, storing your progress through our games, enabling social network integration, providing customer service support when you need it. We also use your information in connection with our advertising purposes. For more information, see below. We may also use your information for research or analysis. Finally, there may also be other reasons why we need to use your information, such as maintaining records or protecting our legal rights. A more detailed description of how we use your information is set out below.

Providing and optimising our Services

We use your information to enable us to provide our games, websites, advertising and other Services to you and to optimise all of them so that we can make them the best they can be for you and all our players. This will include use and analysis of aggregated data to make sure that our games work properly on all devices and are as enjoyable as they can be for all our players.

Chat

Some of our Services include player-to-player chat functionality. Users of our chat service(s) can send messages to other registered players. Your messages may be used and stored by us: (a) to convey your messages; and (b) to enable you and your recipients to view your message history. We reserve the right to review the content of your messages, including by using automatic filters, to ensure that you are complying with our terms of service and we also reserve the right to prevent your use of our chat service(s) or to block the sending of any message for any reason. We will not use the content of your chat messages for any other purpose.

Marketing and promotion

We may use your information to send you direct marketing and promotional materials relating to our Services or related products, for example sending you communications (including by email) for these purposes on our, or a third party’s behalf. For example, we might send you information about merchandise relating to our Services. If you do not want us to use your information in this way please let us know by contacting us at support@voodoo.io.

Customer Services

We use your information, including data that you provide directly to us in communications, to provide you with support through our customer service channels when you need it. We may also use it to contact you, for example as part of customer service or to send you updates about our games and services. Finally, we will use your data to manage your account and relationship with us and improving your experience when you use our Services.

Analytics and Research

We use analytics tools, which may include third party analytics tools, to collect information about how you play our games and those tools may use cookies or other similar tracking technologies. We may use your data to carry out research, surveys or to engage directly with you, for example by sending you communications (including by email), for these purposes. We may also create reports, analysis or similar services for use by us for the purposes of research or business intelligence, for example to track potential problems or trends with our Services, or to test out new game features and content.

Social and Community

We have a number of social and community channels which we use to communicate with our players. If you choose to interact with those channels, we will receive your information and use it in connection with those channels, including to engage with you, and we may also republish your posts to those channels.

Competitions

From time to time we may run competitions, including in games and via our community channels. Those competitions will have their own specific terms and conditions, but we will use personal information that you provide in relation to that competition for the purposes of running the competition.We may also publish the names and entries of competition entrants.

Cheating, crime and fraud prevention

We may use personal data that we hold to prevent cheating, crime or fraud. We may also need to pass your information to fraud prevention agencies and other organisations involved in crime and fraud prevention, such as the police.

Legal uses

We may use your data as required or permitted by any applicable law. As a specific example, if you live in the European Union, this would include for the purpose of providing VAT invoices where we are required to do so by law or where requested by you.

5 Advertising and promotion

What adverts you may see as a result of playing our games or using our Services

We promote our own games and services in a number of ways. That might include cross promoting one of our games while you are playing a different game of ours. It might also include advertising our games within websites and other media published by others, or sending you marketing materials by email.

Like many companies, we may use information about you in connection with advertising. This includes using information that we have about you, such as the games you play, to try to make sure you only see advertising that might be of interest to you. For example, if we know you play Candy Crush Saga, we might suggest that you continue the adventure in Candy Crush Soda Saga by showing you an ad for Candy Crush Soda Saga. We may also use the information that we have about you to help a third party deliver advertisements that are tailored to you based on a determination of your characteristics or interests by us based on the information that we have about you.

As well as advertisements for our own goods and services, we may also facilitate third parties to optimise their advertising of their own goods and services by helping them to target those ads to those that might be most interested in them. These advertisements may be shown within our games or within websites and other media which are published by others and optimised by us or using our information. This is done using anonymised information and we won’t hand over or sell any of your personal information to third party advertisers.

We will also use information about your particular device in order to show you adverts which work properly on that device, or to measure the effectiveness of our ad campaigns and we may also retain information about how many times you have seen a particular advertisement so we can stop you from seeing it too many times.

Advertising partners

When we advertise our games in games or media published by other companies, we may use various third party advertising partners, including ad exchanges, ad networks and ad servers.

Our advertising partners help us to serve advertisements to you. Some of this technology also helps us to make decisions as to whether or not to show a particular advert to you – for example, by informing us that you have shown an interest in similar goods or services previously.

Our advertising partners may collect information about you when you use our services or the services of others. This Privacy Policy does not apply to the collection of your information by our advertising partners, and we therefore recommend that you review their privacy policies for more information. You can find out more in “How to adjust your preferences” below.

How do ad identifiers, cookies and similar technologies work?

Advertisers on mobile devices sometimes use advertising (or ad) identifiers to enable and optimise their advertising. Ad identifiers are non-permanent, non-personal device identifiers such as the Android advertising ID and/or Apple’s ID for advertising, or IDFA. See below for you can reset these identifiers or indicate your preference that they should not be used for advertising purposes.

We and third parties who advertise on our Services may use cookies or similar technologies such as tracking pixels in their advertising. These technologies operate either by: (i) placing a small file which stores some information on your computer or mobile device; and/or (ii) by accessing information on your device. These technologies allow the collection of data, such as your device’s model, operating system and screen size, the other applications installed on your device, and information about how you use our services.

How to adjust your preferences

You can choose to prevent your device’s ad identifier being used for interest based advertising, or you can reset your device’s ad identifier, by changing the settings of your device. Settings vary from device to device but you will usually find the ad identifier settings under “privacy” or “ads” in your device’s settings.

You can disable cookies through your web or phone browser settings.

If you live in the USA, Europe or Canada, you can also opt-out from the collection of your information by some or all of our advertising partners who participate in the Digital Advertising Alliance by visiting http://www.aboutads.info/choices/ (for US users), http://www.youronlinechoices.com/ (for European users) or http://youradchoices.ca/choices (for Canadian users). You can also click on the AdChoices logo within an advertisement (where available).

Although we and our advertising partners use ad identifiers, cookies and similar technologies to enable us and third parties to provide more relevant advertising to you, these technologies also:

allow us to recognise you and your device;
allow our Services to interact with a third party social network or platform (where you have chosen to allow such interaction);
allow our payment processors to process your payment instructions; and
enable us and third parties to provide you with more customized services, for example to provide our services in the correct language.
As a result, if you choose to adjust your preferences in order to prevent these technologies, you may find that you cannot enjoy the full use of our services.

Please note that adjusting your preferences as described in this section does not mean you will no longer receive adverts, it only means the adverts you do see will be less relevant to your interests.

Alternatively, if you do not wish us to collect data as described in this section, you should stop using our services.

6 Who else can see your information?

In addition to the other disclosures described in this privacy policy, we will also share your information as described in this section.

 

We will share your information:

where we have your consent to do so (including as set out in this Privacy Policy);
as reasonably necessary in order to provide the Services to you (for example, by providing your personal information to suppliers we may use to fulfil the Services or to communicate with you);
where it is necessary to carry out your instructions (for example, to process a payment instruction we must provide your personal information to our payment processors);
where your data is on an anonymous and aggregated basis, meaning you could not be personally identified from it;
when you submit information as part of a competition or otherwise interact with our social media channels (for example, by providing a testimonial about one of our Services) we may publish the information you submitted, alongside your name and home country on our website(s) and social media channels;
as we reasonably believe is permitted by law or regulation or as is necessary to comply with any legal obligation, or in order to enforce or apply our terms and conditions and/or any other agreement with you; or to protect the rights, property, or safety of Voodoo, our users, or others;
if there is a sale of the assets of Voodoo or corporate restructuring, or as a result of a change of control of Voodoo or one of its group companies, or in preparation of any of these events. Any third party to which Voodoo transfers of sells Voodoo’s assets will have the right to continue to use the personal and other information that you provide to us in the manner set out in this Privacy Policy.
Our games and our websites include social features that enable you to interact with other players. These features will enable other players to see your username, your avatar and your progress through our games and other game play related information such as high scores. Depending on your settings in your social network or Voodoo account, other information in your profile may be shared with other players. Also, other players will be able to see anything that you post to any public page, forum or other similarly open feature in our games, websites, community channels or other Services.

We may share your information with our service providers if necessary to enable them to provide services to us.

7 Your rights in relation to your information

You have certain rights in connection with your personal information and how we handle it. Details of those rights and how to exercise them are contained in this section.

 

You have certain rights in relation to the personal information that we hold about you. Those rights include:

Right of access. You have a right to know what information we hold about you and in some cases to have the information communicated to you. If you wish to exercise this right please contact us letting us know that you wish to exercise your right of access and what information in particular you would like to receive. We reserve the right to ask for reasonable evidence to verify your identity before we provide you with any information, and the law permits us to charge a fee for exercising this right of access. Please note that we may not be able to provide all the information you ask for, for instance if the information includes personal information about another person. Where we are not able to provide you with information that you have asked for, we will endeavour to tell you why. We will try to respond to any request for a right of access as soon as possible, but we will always do so within 40 days of receipt of your request, evidence of ID and any applicable fee.
Right to correct personal information. We try to keep the information that we hold about you accurate and up to date. Should you realise that any of the information that we hold about you is incorrect, please let us know at support@voodoo.io and we will correct it as soon as we can.
Data deletion. In some circumstances you have a right to have some of the personal information that we hold about you deleted. Should you wish to have any information about you deleted, please contact us using the information below. Please note that to ensure that we do not collect any further information you should also delete our games from your mobile devices and clear our cookies from any device where you have played our games in a web browser. Where we delete personal information about you, we may still retain some or all of that information for other purposes such as maintaining financial records, protecting or enforcing legal rights, maintaining marketing suppression lists or for technical reasons such as maintaining technical security or our database integrity. We may also retain your information in an anonymised form.
Account deactivation. If you have created an account with us, you can ask us to deactivate that account by contacting us using the information below. We may ask you for other information before we are able to deactivate your account. That information might include information about your mobile device. Other steps you should take should you wish to completely deactivate your account include disassociating your Facebook account from our mobile games and deleting our games from your mobile devices. Please note that if you completely deactivate your account all your progress in our games and any unspent virtual items, such as Gold Bars or Boosters will be lost and we may not be able to restore them in the future.
Direct marketing opt out. If you have elected to receive direct marketing communications from us, you can change your mind at any time by following the opt out link in any marketing communication that is sent to you. If you have elected to receive more than one type of marketing communications from us, you may need to opt out of all of them individually. It may take a few days for us to update our records before any opt out is effective.
There are other ways in which you can control the personal information that we collect about you. For instance, you could disconnect your Facebook or other social network account from our games, or delete our games from your mobile device. You could also reset your mobile advertising identifier or limit ad tracking altogether using the settings in your phone. Finally, you could limit or prevent the use of cookies in your web browsers.

8 How long do we keep your information and where do we store it?

We will keep your information for as long as we consider necessary for the purposes described above, or as long as the law allows. We store your information on servers that we control which are primarily based in the European Union. We may also use servers that are located outside the European Union and we may engage partners outside the European Union who will also process and store your information to provide services to us.

9 Links to third party websites and services

We may provide links to third party applications, services or websites from our Services (including advertising that may link to a third party). You understand that when you click on these links any data which you provide afterwards is subject to that third party’s privacy policy and not to ours. We can take no responsibility for the content, safety, privacy or security of any third party application, service or website.

10 Changes to the policy

The privacy policy and this summary may change over time. We will post the updated privacy policy on this page. Please come back to this page every now and then to make sure you are familiar with the latest version. Any new policy will be effective from the date it is published by us.


### Reading your contacts

This one is interesting because this is data not just about you but about the people you know too - with this data advertisers can piece together the identities of your friends even if they haven't installed their app

In [173]:
df[df['read your contacts']].title

com.oneapp.max.cleaner.booster                      MAX Cleaner - Phone Cleaner & Antivirus
com.psafe.msuite                          dfndr security: antivirus, anti-hacking & cleaner
com.enflick.android.TextNow                                     TextNow - free text + calls
com.facebook.katana                                                                Facebook
com.ehawk.antivirus.applock.wifi          Virus Cleaner ( Hi Security ) - Antivirus, Boo...
com.facebook.orca                                  Messenger – Text and Video Chat for Free
com.zhiliaoapp.musically                                                         musical.ly
com.whatsapp                                                             WhatsApp Messenger
com.apusapps.browser                        APUS Browser - Fast Download & Private & Secure
com.ubercab                                                                            Uber
com.amazon.mShop.android.shopping                                           Amaz

In [177]:
df.loc['com.contextlogic.wish'].privacy_url

'http://www.wish.com/privacy_policy'

Cool, so these guys link directly to a privacy policy - I wonder what it says...

Along with the ususal 'we'll use your data to provide ads' stuff they also look to be selling you data in aggregate
```
Anonymous & Aggregate Information
We collect some information on an anonymous basis. We also may anonymize the personal information we collect about you.
We obtain aggregate data by combining anonymous data that meet certain criteria into groups.
We may share aggregate or anonymous information in various formats with trusted non-ContextLogic entities, and may work with those entities to do research and provide products and services.
```

Better hope they do a good job of that anonymisation :) 

### Companies without a privacy policy?

There were some companies where my `get_privacy_policy` function failed - let's investigate

In [183]:
df[df.privacy_url == '']

,permissions,playstoreUrl,title,approximate location (network-based),precise location (GPS and network-based),read the contents of your USB storage,modify or delete the contents of your USB storage,view Wi-Fi connections,read phone status and identity,view network connections,...,bind to an accessibility service,delete other apps' data,read historical network usage,control media playback and metadata access,send package removed broadcast,manage document storage,read terms you added to the dictionary,add words to user-defined dictionary,cnt,privacy_url
com.tastypill.newbottle,[{'permission': 'approximate location (network...,https://play.google.com/store/apps/details?id=...,Impossible Bottle Flip,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,20.0,
com.itchmedia.ta3,[{'permission': 'approximate location (network...,https://play.google.com/store/apps/details?id=...,Baseball Boy!,True,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,14.0,
com.bdj.vortexDroid,[{'permission': 'approximate location (network...,https://play.google.com/store/apps/details?id=...,Rolly Vortex,True,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,14.0,
com.dvloper.granny,[{'permission': 'read the contents of your USB...,https://play.google.com/store/apps/details?id=...,Granny,False,False,True,True,False,False,True,...,False,False,False,False,False,False,False,False,10.0,
com.PoxelStudios.DudeTheftAuto,[{'permission': 'read the contents of your USB...,https://play.google.com/store/apps/details?id=...,Dude Theft Wars: Open World Sandbox Simulator ...,False,False,True,True,True,False,True,...,False,False,False,False,False,False,False,False,10.0,
com.squidward.sponge.neighbor,"[{'permission': 'view network connections', 'd...",https://play.google.com/store/apps/details?id=...,Hello Squidward. Sponge Bob's Neighbor 3D,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,4.0,


These all seem to be games, the `Impossible bottle flip` asks for permissions to 10 resources and doesn't say what it's going to do with that data

In [199]:
bottle = df.loc['com.tastypill.newbottle']
bottle[bottle!=False]

permissions                                          [{'permission': 'approximate location (network...
playstoreUrl                                         https://play.google.com/store/apps/details?id=...
title                                                                           Impossible Bottle Flip
approximate location (network-based)                                                              True
precise location (GPS and network-based)                                                          True
read the contents of your USB storage                                                             True
modify or delete the contents of your USB storage                                                 True
view Wi-Fi connections                                                                            True
read phone status and identity                                                                    True
view network connections                                                 